In [20]:
import pandas as pd
import numpy as np
import networkx as nx
import random
from scipy.spatial.distance import euclidean
from itertools import combinations
import plotly.graph_objects as go
import webbrowser

In [21]:
df = pd.read_csv("tum_sentetik2.csv")
df['Kategori'] = df['Mağaza'].apply(lambda x: x.split('_')[0])

available_products = df['Ürün'].unique()
product_to_category = df.groupby('Ürün')['Kategori'].first().to_dict()

# Mağaza kategorileri için renkler 
kategori_renkleri = {
    "Gida": "orange",
    "Kozmetik": "purple",
    "Giysi": "blue",
    "Elektronik": "red",
    "Ev": "green"
}

def get_shopping_list(available_products, product_to_category):
    print("\n=== Alışveriş Listesi Oluşturma ===")
    
    shopping_list = []
    print("\nAlışveriş listesine eklemek istediğiniz ürünleri girin.")
    print("Bitirmek için 'bitir' yazın.")
    
    while True:
        product = input("Ürün adı: ").strip()
        if product.lower() == 'bitir':
            break
        if product in available_products:
            shopping_list.append(product)
            print(f"{product} listeye eklendi.")
        else:
            print(f"Hata: '{product}' veri setinde bulunmuyor. Lütfen geçerli bir ürün girin.")
    
    if not shopping_list:
        raise ValueError("Alışveriş listesi boş olamaz! Lütfen en az bir ürün ekleyin.")
    
    print("\nOluşturulan Alışveriş Listesi:", shopping_list)
    return shopping_list

# Alışveriş listesini al
print("Merhaba! Size en uygun alışveriş rotasını bulacağım.")
shopping_list = get_shopping_list(available_products, product_to_category)

# Ürün kategorilerini oluştur
item_categories = {item: product_to_category.get(item, "Bilinmeyen") for item in shopping_list}

# Ürünleri içeren mağazaları bul
filtered_stores = df[df['Ürün'].isin(shopping_list)][['Mağaza', 'X_Koordinat', 'Y_Koordinat', 'Kategori']].drop_duplicates(subset=['Mağaza'])
filtered_stores = filtered_stores.dropna(subset=['X_Koordinat', 'Y_Koordinat'])

valid_stores = set(filtered_stores['Mağaza'])
store_products = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].groupby('Mağaza')['Ürün'].apply(set).to_dict()
product_prices = df[df['Ürün'].isin(shopping_list) & df['Mağaza'].isin(valid_stores)].set_index(['Mağaza', 'Ürün'])['Fiyat'].to_dict()

# Ürün-Mağaza eşleştirmesi
item_to_stores = {item: set() for item in shopping_list}
for store in filtered_stores['Mağaza']:
    store_category = store.split('_')[0]
    for item in store_products.get(store, set()):
        if item in shopping_list and item_categories[item] == store_category:
            item_to_stores[item].add(store)

# "Ev" düğümünü ekle (kullanıcıdan koordinat al)
print("\nŞimdi 'Ev' noktasının koordinatlarını belirleyeceğiz (0-100 arası).")
while True:
    try:
        home_x = float(input("Ev'in X koordinatı (0-100): "))
        home_y = float(input("Ev'in Y koordinatı (0-100): "))
        if 0 <= home_x <= 100 and 0 <= home_y <= 100:
            break
        else:
            print("Hata: Koordinatlar 0 ile 100 arasında olmalı!")
    except ValueError:
        print("Hata: Lütfen geçerli bir sayı girin!")
home_node = "Ev"
print(f"Ev koordinatları: ({home_x}, {home_y})")

Merhaba! Size en uygun alışveriş rotasını bulacağım.

=== Alışveriş Listesi Oluşturma ===

Alışveriş listesine eklemek istediğiniz ürünleri girin.
Bitirmek için 'bitir' yazın.


Bant listeye eklendi.
Sabun listeye eklendi.
Hırka listeye eklendi.
Elbise listeye eklendi.
Bot listeye eklendi.
Süt listeye eklendi.
Peynir listeye eklendi.
Tablet listeye eklendi.
Oje listeye eklendi.

Oluşturulan Alışveriş Listesi: ['Bant', 'Sabun', 'Hırka', 'Elbise', 'Bot', 'Süt', 'Peynir', 'Tablet', 'Oje']

Şimdi 'Ev' noktasının koordinatlarını belirleyeceğiz (0-100 arası).
Ev koordinatları: (0.0, 0.0)


In [22]:
# Graph oluştur ve "Ev"i ekle
G_aco = nx.Graph()
G_aco.add_node(home_node, pos=(home_x, home_y), kategori="Ev")
for _, row in filtered_stores.iterrows():
    G_aco.add_node(row['Mağaza'], pos=(row['X_Koordinat'], row['Y_Koordinat']), kategori=row['Kategori'])


all_nodes = list(G_aco.nodes())
for (m1, pos1), (m2, pos2) in combinations([(n, G_aco.nodes[n]['pos']) for n in all_nodes], 2):
    distance = euclidean(pos1, pos2)
    travel_time = distance / 60
    G_aco.add_edge(m1, m2, weight=travel_time)

store_list = list(G_aco.nodes())

### ACO FUNCTIONS

In [23]:
# Maliyet hesaplama fonksiyonu (ürün fiyatı + ulaşım maliyeti)
def calculate_cost(route, graph, product_prices, shopping_list, store_products):
    total_cost = 0
    collected_items = set()
    item_sources = {}
    
    for store in route:
        if store != "Ev":
            available_items = store_products.get(store, set()) & set(shopping_list) - collected_items
            for item in available_items:
                price = product_prices.get((store, item), float('inf'))
                if price < float('inf'):
                    item_sources[item] = (store, price)
                    collected_items.add(item)
                    total_cost += price
    
    for i in range(len(route) - 1):
        store1, store2 = route[i], route[i + 1]
        travel_cost = graph[store1][store2]['weight'] * 100
        total_cost += travel_cost
    
    if collected_items != set(shopping_list):
        return float('inf'), item_sources
    
    return total_cost, item_sources

# ACO Algoritması
def ant_colony_optimization(graph, store_list, product_prices, shopping_list, store_products, item_to_stores, num_ants=10, num_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, Q=100):
    pheromone = {edge: 1.0 for edge in graph.edges()}
    best_route = None
    best_cost = float('inf')
    best_sources = {}

    print("\nOptimizasyon başlıyor...")
    for iteration in range(num_iterations):
        all_routes = []
        all_costs = []
        
        for _ in range(num_ants):
            route = ["Ev"]
            collected_items = set()
            
            while collected_items != set(shopping_list):
                current_store = route[-1]
                missing_items = set(shopping_list) - collected_items
                candidates = [s for s in store_list if s != "Ev" and s not in route and any(item in store_products.get(s, set()) for item in missing_items)]
                
                if not candidates:
                    break
                
                probabilities = []
                for candidate in candidates:
                    travel_weight = graph[current_store][candidate]['weight']
                    tau = pheromone.get((current_store, candidate), pheromone.get((candidate, current_store), 1.0)) ** alpha
                    items_here = store_products.get(candidate, set()) & missing_items
                    avg_price = sum(product_prices.get((candidate, item), float('inf')) for item in items_here) / max(1, len(items_here))
                    eta = (1.0 / (travel_weight + avg_price / 100)) ** beta
                    probabilities.append(tau * eta)
                
                total_prob = sum(probabilities)
                if total_prob > 0:
                    probabilities = [p / total_prob for p in probabilities]
                    next_store = random.choices(candidates, weights=probabilities, k=1)[0]
                else:
                    next_store = random.choice(candidates)
                
                route.append(next_store)
                collected_items |= store_products.get(next_store, set()) & set(shopping_list)
            
            route.append("Ev")
            cost, sources = calculate_cost(route, graph, product_prices, shopping_list, store_products)
            all_routes.append(route)
            all_costs.append((cost, sources))
        
        for route, (cost, sources) in zip(all_routes, all_costs):
            if cost < best_cost:
                best_route = route
                best_cost = cost
                best_sources = sources
                print(f"Iteration {iteration}: Best Cost = {best_cost:.2f}")
        
        for edge in pheromone:
            pheromone[edge] *= (1 - evaporation_rate)
        for route, (cost, sources) in zip(all_routes, all_costs):
            if cost != float('inf'):
                delta = Q / cost
                for i in range(len(route) - 1):
                    edge = (route[i], route[i + 1])
                    if edge in pheromone:
                        pheromone[edge] += delta
                    elif (route[i + 1], route[i]) in pheromone:
                        pheromone[(route[i + 1], route[i])] += delta
    
    return best_route, best_cost, sum(price for _, (store, price) in best_sources.items()), best_sources

In [24]:
# ACO’yu Çalıştır
optimal_route, total_cost, total_price, item_sources = ant_colony_optimization(
    G_aco, store_list, product_prices, shopping_list, store_products, item_to_stores
)

from IPython.display import display
display(optimal_route)

print("+++++++++++++++++++++++")

# Sonuçları Yazdır
print("\n=== ACO Sonuçları ===")
print("Optimal Rota:")
for i, store in enumerate(optimal_route, 1):
    print(f"  {i}. {store}")
print(f"\nToplam Maliyet (Ulaşım + Ürün): {total_cost:.2f} TL")
print(f"Toplam Ürün Fiyatı: {total_price:.2f} TL")
print("\nÜrün Kaynakları:")
for item in shopping_list:
    if item in item_sources:
        store, price = item_sources[item]
        print(f"  {item}: {store} (Fiyat: {price:.2f} TL)")
    else:
        print(f"  {item}: Bulunamadı")
missing_items = set(shopping_list) - set(item_sources.keys())
if not missing_items:
    print("\nTüm ürünler başarıyla alındı!")
else:
    print("\nEksik ürünler:", missing_items)
print("\nSonuçlar bir HTML dosyasında görselleştirildi ve tarayıcınızda açılacak.")

# Görselleştirme 
pos_aco = nx.get_node_attributes(G_aco, 'pos')
node_x = [pos_aco[node][0] for node in G_aco.nodes()]
node_y = [pos_aco[node][1] for node in G_aco.nodes()]
node_color = ["black" if node in optimal_route and node != "Ev" else "aqua" if node == "Ev" else kategori_renkleri.get(G_aco.nodes[node]['kategori'], 'gray') for node in G_aco.nodes()]
node_text = [node if node in optimal_route else "" for node in G_aco.nodes()]

edge_x = []
edge_y = []
for i in range(len(optimal_route) - 1):
    x0, y0 = pos_aco[optimal_route[i]]
    x1, y1 = pos_aco[optimal_route[i + 1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

fig = go.Figure()
fig.add_trace(go.Scatter(x=edge_x, y=edge_y, line=dict(width=2, color='gray'), mode='lines', name="Rota"))
fig.add_trace(go.Scatter(
    x=node_x, y=node_y, mode='markers+text',
    marker=dict(size=10, color=node_color),
    text=node_text, textposition="top center", hoverinfo="text"
))
fig.update_layout(
    title="Optimal Alışveriş Rotası ve Ürün Seçimi (ACO Algoritması)",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)
fig.write_html("aco_optimized.html")
webbrowser.open("aco_optimized.html")


Optimizasyon başlıyor...
Iteration 0: Best Cost = 4310.53
Iteration 0: Best Cost = 3312.08
Iteration 0: Best Cost = 3234.86
Iteration 0: Best Cost = 2576.28
Iteration 0: Best Cost = 2489.65
Iteration 1: Best Cost = 2428.25
Iteration 1: Best Cost = 2422.53
Iteration 4: Best Cost = 2380.48
Iteration 6: Best Cost = 2235.03
Iteration 7: Best Cost = 2172.80
Iteration 8: Best Cost = 2090.66
Iteration 9: Best Cost = 1957.90
Iteration 10: Best Cost = 1903.94


['Ev',
 'Gida_Magaza_16',
 'Gida_Magaza_37',
 'Kozmetik_Magaza_32',
 'Ev_Magaza_40',
 'Giysi_Magaza_24',
 'Ev_Magaza_27',
 'Giysi_Magaza_22',
 'Elektronik_Magaza_13',
 'Ev']

+++++++++++++++++++++++

=== ACO Sonuçları ===
Optimal Rota:
  1. Ev
  2. Gida_Magaza_16
  3. Gida_Magaza_37
  4. Kozmetik_Magaza_32
  5. Ev_Magaza_40
  6. Giysi_Magaza_24
  7. Ev_Magaza_27
  8. Giysi_Magaza_22
  9. Elektronik_Magaza_13
  10. Ev

Toplam Maliyet (Ulaşım + Ürün): 1903.94 TL
Toplam Ürün Fiyatı: 1218.68 TL

Ürün Kaynakları:
  Bant: Ev_Magaza_27 (Fiyat: 121.04 TL)
  Sabun: Ev_Magaza_40 (Fiyat: 220.33 TL)
  Hırka: Giysi_Magaza_24 (Fiyat: 260.26 TL)
  Elbise: Giysi_Magaza_22 (Fiyat: 65.51 TL)
  Bot: Giysi_Magaza_22 (Fiyat: 92.61 TL)
  Süt: Gida_Magaza_37 (Fiyat: 62.63 TL)
  Peynir: Gida_Magaza_16 (Fiyat: 73.28 TL)
  Tablet: Elektronik_Magaza_13 (Fiyat: 268.61 TL)
  Oje: Kozmetik_Magaza_32 (Fiyat: 54.41 TL)

Tüm ürünler başarıyla alındı!

Sonuçlar bir HTML dosyasında görselleştirildi ve tarayıcınızda açılacak.


True